# Overview of Arangopipe

This notebook provides an overview of **Arangopipe**, a component of ArangoDB for managing metadata from machine learning pipelines. Arangopipe has two API's:
1. **arangopipe_api**
2. **arangopipe_admin_api**
**arangopipe_api** is the set of API used for machine learning metadata management. **arangopipe_admin_api** is the API used to provision users into **Arangopipe**. The following notebook illustrates both these API's. We will illustrate this with a machine learning model to predict house prices. The data is available in the UCI machine learning repository.

## Administrative Pre-requisites

The first task is to setup a project in Arangopipe. A project could require multiple models. For illustrative purposes, we will create a single model. Setting up a project is accomplished using the **arangopipe_admin_api**

### Install ArangoDB
Install arangodb. The docker version of arangodb can be installed by:

`docker run -p 8529:8529 -e ARANGO_ROOT_PASSWORD=openSesame arangodb/arangodb:3.4.6`

### Install Arangopipe

In [1]:
import sys
!{sys.executable} -m pip install python-arango

In [4]:
!{sys.executable} -m pip install -i https://test.pypi.org/simple/ arangopipe

Looking in indexes: https://test.pypi.org/simple/


### Create a Project
Use the admministrative API to register a project with Arangopipe

In [5]:
from arangopipe.arangopipe_admin_api import ArangoPipeAdmin
proj_info = {"name": "Housing Price Estimation Project "}
admin = ArangoPipeAdmin()
proj_reg = admin.register_project(proj_info)

### Associate Model with Project
This pipeline is going to determine the best regression model to use for the project. We will conduct this experiment with hyperopt. First, however we link the the model developed in this pipeline with the project

In [6]:
from arangopipe.arangopipe_api import ArangoPipe

ap = ArangoPipe()
model_info = {"name": "hyper-param-optimization",  "type": "hyper-opt-experiment"}
model_reg = ap.register_model(model_info, "Housing Price Estimation Project")

## Pipeline Development
This notebook illustrates the process of storing pipeline metadata while executing a machine learning pipeline. The objective with this experiment is to determine the best model for the dataset using the **Hyperopt** library. After conducting the experiments, the result is tagged and stored in **Arangopipe**. 

### Read Data

In [7]:
import pandas as pd
fp = "cal_housing.csv"
df = pd.read_csv(fp)

### Register the Dataset

In [8]:
ds_info = {"name" : "california-housing-dataset",\
            "description": "This dataset lists median house prices in Califoria. Various house features are provided",\
           "source": "UCI ML Repository" }
ds_reg = ap.register_dataset(ds_info)

### Register the Featureset Generated from the Dataset
A log transformation is required for the median-house value. The feature set generated from the dataset is registered with **Arangopipe**. Note that the featureset is linked to the dataset using the dataset registration obtained from the previous step

In [9]:
import numpy as np
df["medianHouseValue"] = df["medianHouseValue"].apply(lambda x: np.log(x))
featureset = df.dtypes.to_dict()
featureset = {k:str(featureset[k]) for k in featureset}
featureset["name"] = "log_transformed_median_house_value"
fs_reg = ap.register_featureset(featureset, ds_reg["_key"])

### Run the Experiment

In [10]:
from sklearn.model_selection import train_test_split
preds = df.columns.tolist()
preds.remove("medianHouseValue")
X = df[preds]
Y = df["medianHouseValue"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

**Set up the Hyperopt Experiment**

Define the hyper-opt space. In this case, this represents the various models and their associated parametrizations

In [11]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn import neighbors
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error

hyper_param_options =     [{
        'type': 'lasso',
        'alpha': hp.uniform('alpha', 0.0, 1)
    },

    {
        'type': 'randomforest',
        'max_depth': hp.choice('max_depth', range(1,5)),
        'max_features': hp.choice('max_features', range(1,8)),
        'n_estimators': hp.choice('n_estimators', range(1,50))
    },
    {
        'type': 'knn',
        'n_neighbors': hp.choice('knn_n_neighbors', range(1,20))
    }
]
space = hp.choice('regressor_type', hyper_param_options)

def hyperopt_train_test(params):
    regressor_type = params['type']
    del params['type']
    if regressor_type == 'lasso':
        reg = linear_model.Lasso(**params)
    elif regressor_type == 'randomforest':
        reg = RandomForestRegressor(**params)
    elif regressor_type == 'knn':
        reg = neighbors.KNeighborsRegressor(**params)
    else:
        return 0
    reg.fit(X_train, y_train)
    ytest_pred = reg.predict(X_test)
    return mean_squared_error(y_test, ytest_pred)

/home/admin2/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


**Run the Hyperopt Experiment**

In [12]:
count = 0
best = 100
def f(params):
    global best, count
    count += 1
    rmse = hyperopt_train_test(params.copy())
    if rmse < best:
        print ('new best:', rmse, 'using', params['type'])
        best = rmse
    if count % 250 == 0:
        print ('iters:', count, ', acc:', rmse, 'using', params)
    return {'loss': rmse, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space, algo=tpe.suggest, max_evals=500, trials=trials)
print ('best:', best)

new best:                                            
0.154261903392289                                    
using                                                
randomforest                                         
new best:                                                                     
0.11373465186192634                                                           
using                                                                         
lasso                                                                         
new best:                                                                        
0.11372726080280238                                                              
using                                                                            
lasso                                                                            
new best:                                                                        
0.11372691580042597                                       

## Convert Hyperopt Space to JSON

In [13]:
import jsonpickle
import uuid
ruuid = str(uuid.uuid4().int)
frozen_space = jsonpickle.encode(space)
model_params = {"hyper-param-space": frozen_space, "run_id": ruuid}

## Store Results in Arangopipe
Note that we are tagging the run so that we can look up this run by the tag if we need to retrieve it from storage

In [14]:

import datetime

model_perf = {"best": jsonpickle.encode(best), "run_id": ruuid, "timestamp": str(datetime.datetime.now())}
run_info = {"dataset" : ds_reg["_key"],\
                    "featureset": fs_reg["_key"],\
                    "run_id": ruuid,\
                    "model": model_reg["_key"],\
                    "model-params": model_params,\
                    "model-perf": model_perf,\
                    "tag": "Housing-Price-Hyperopt-Experiment",\
                    "project": "Housing Price Estimation Project"}
ap.log_run(run_info)


## What was the best the model from the previous run?
The tag (Housing-Price-Hyperopt-Experiment) that we applied while logging the previous experiment can be used to retrieve the results associated with the previous run. For example, we may be interested in the best model and its parameters from the experiment we just conducted.

In [15]:
mp = ap.lookup_modelperf("Housing-Price-Hyperopt-Experiment")

In [16]:
mp["best"]

'{"alpha": 3.8800929466159645e-05, "regressor_type": 0}'

In [13]:
from arangopipe.arangopipe_config import ArangoPipeConfig
apc = ArangoPipeConfig()

In [17]:
apc.set_dbconnection(root_user = 'root2test', root_user_password = 'admin123')
ap = ArangoPipe(config = apc, persist = False)

DatabaseListError: [HTTP 401][ERR 11] not authorized to execute this request